In [1]:
# 1. Install the missing storage library first
!pip install google-cloud-bigquery-storage

# 2. Attempt to satisfy the strictest requirements (Colab and Gradio)
!pip install \
    "google-auth==2.38.0" \
    "requests==2.32.4" \
    "pydantic<2.12" \
    "rich<14" \
    "fsspec==2025.3.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.7/303.7 kB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninsta

In [2]:
!pip install -q roboflow pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 40.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.9 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.57.0 requires google-auth[requests]<3.0.0,>=2.46.0, but you have google-auth 2.38.0 which is incompatible.
google-cloud-aiplatform 1.133.0 requires google-auth<3.0.0,>=2.47.0, but you have google-auth 2.38.0 which is incompatible.
google-adk 1.22.1 requires google-auth>=2.47.0, but you have google-auth 2.38.0 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyt

In [3]:
from roboflow import Roboflow

rf = Roboflow(api_key="1706kXS5zd9ApFxEBXnK")
project = rf.workspace("qweroiuyt").project("rcnn-microplastic-itjer")
version = project.version(1)
dataset = version.download("yolov11")

DATASET_ROOT = dataset.location
print(DATASET_ROOT)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to RCNN-microplastic-1 in yolov11:: 100%|██████████| 29513/29513 [00:03<00:00, 9616.95it/s] 


/kaggle/working/RCNN-microplastic-1


In [4]:
import os
import yaml
import torch
import torchvision
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [5]:
class YOLODataset(Dataset):
    def __init__(self, root, split):
        self.img_dir = os.path.join(root, split, "images")
        self.lbl_dir = os.path.join(root, split, "labels")

        self.images = sorted(os.listdir(self.img_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        lbl_path = os.path.join(
            self.lbl_dir,
            img_name.replace(".jpg",".txt").replace(".png",".txt")
        )

        image = Image.open(img_path).convert("RGB")
        w, h = image.size

        boxes = []
        labels = []

        if os.path.exists(lbl_path):
            with open(lbl_path) as f:
                for line in f.readlines():
                    cls, x, y, bw, bh = map(float,line.split())

                    x1 = (x - bw/2)*w
                    y1 = (y - bh/2)*h
                    x2 = (x + bw/2)*w
                    y2 = (y + bh/2)*h

                    boxes.append([x1,y1,x2,y2])
                    labels.append(int(cls)+1)

        target = {
            "boxes": torch.tensor(boxes,dtype=torch.float32),
            "labels": torch.tensor(labels,dtype=torch.int64)
        }

        image = torchvision.transforms.ToTensor()(image)

        return image, target

In [6]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_ds = YOLODataset(DATASET_ROOT,"train")
val_ds   = YOLODataset(DATASET_ROOT,"valid")

train_loader = DataLoader(train_ds,batch_size=4,
                          shuffle=True,collate_fn=collate_fn)

val_loader = DataLoader(val_ds,batch_size=4,
                        shuffle=False,collate_fn=collate_fn)

In [7]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

num_classes = 2

model = fasterrcnn_resnet50_fpn(weights="DEFAULT")

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(
    in_features,num_classes
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:00<00:00, 199MB/s] 


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [8]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.AdamW(
    params,
    lr=1e-4,
    weight_decay=1e-4
)

In [9]:
from tqdm import tqdm

def train_one_epoch(model,loader):
    model.train()
    total_loss = 0

    for images,targets in tqdm(loader):
        images = [img.to(device) for img in images]
        targets = [{k:v.to(device) for k,v in t.items()} for t in targets]

        loss_dict = model(images,targets)
        loss = sum(loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss/len(loader)

In [10]:
NUM_EPOCHS = 20
best_loss = float("inf")

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}")

    train_loss = train_one_epoch(model,train_loader)

    print("Train Loss:",train_loss)

    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model.state_dict(),"best.pt")


Epoch 1


100%|██████████| 2682/2682 [19:11<00:00,  2.33it/s]


Train Loss: 0.3132130391284612

Epoch 2


100%|██████████| 2682/2682 [19:09<00:00,  2.33it/s]


Train Loss: 0.2899460997279681

Epoch 3


...

Train Loss: 0.053871682906866426

Epoch 19


100%|██████████| 2682/2682 [19:17<00:00,  2.32it/s]


Train Loss: 0.0559686525345632

Epoch 20


100%|██████████| 2682/2682 [19:23<00:00,  2.31it/s]


Train Loss: 0.05081416298840181


In [11]:
model.load_state_dict(torch.load("best.pt"))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(